# Tennis Machine Learning Model: 
Training a model on tennis matches to predict future ones.

## Preprocess data

Read data and set dates to datetime format.

In [515]:
import pandas as pd
from glob import glob

# adjust path or filenames to where you saved the CSVs
files = sorted(glob("MatchCSVs/atp_matches_*.csv"))[:5] # Finds all files matching the pattern, sorts them, and takes the first 2 as an example
dfs = [pd.read_csv(f) for f in files] # Read each CSV file into a DataFrame
df = pd.concat(dfs, ignore_index=True) # Concatenate all DataFrames into one, resetting the index

# Quick peek
print("Rows:", len(df))
print("Columns:", df.columns.tolist())

# Make sure dates are in datetime format (if not already)
if 'tourney_date' in df.columns:
    df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d', errors='coerce')
else:
    # fallback
    df['tourney_date'] = pd.to_datetime(df['date'], errors='coerce')

#print(df['tourney_date'].min(), "->", df['tourney_date'].max())

Rows: 13174
Columns: ['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points']


Drop walkover matches, rows with missing data, and sort chronologically.

In [516]:
# drop problem matches (RET, walkover)
if 'score' in df.columns:
    df = df[~df['score'].str.contains('RET|Walkover|WO|ret', case=False, na=False)] # Remove rows with RET, Walkover, or similar in the score column

# drop rows with missing key data
df = df.dropna(subset=['winner_id','loser_id','tourney_date'])

# sort chronologically (important for time-based features)
# Build a round sort key so finals come last within each tournament
round_order_map = {
    'RR':  5, 
    'R128': 10, 'R64': 20,
    'R32': 30, 'R16': 40,
    'QF': 50, 'SF': 60,        # round robin before knockouts                                          # bronze/3rd place if present
    'F':   70
}
df['round_order'] = df.get('round').map(round_order_map).fillna(999).astype(int)

# Compose sort columns
sort_cols = ['tourney_date']
if 'tourney_id' in df.columns:
    sort_cols.append('tourney_id')
elif 'tourney_name' in df.columns:
    sort_cols.append('tourney_name')

sort_cols.append('round_order')
if 'match_num' in df.columns:
    sort_cols.append('match_num')

# Sort chronologically and within tournament by round
df = df.sort_values(sort_cols).reset_index(drop=True)
df = df.drop(columns=['round_order'])

display(df['round'].iloc[0:27])

0     R32
1     R32
2     R32
3     R32
4     R32
5     R32
6     R32
7     R32
8     R32
9     R32
10    R32
11    R32
12    R16
13    R16
14    R16
15    R16
16    R16
17    R16
18    R16
19    R16
20     QF
21     QF
22     QF
23     QF
24     SF
25     SF
26      F
Name: round, dtype: object

Create a player-event table for rolling stats (i.e. gather all information before a certain match to use in training)
Build a long "events" table where each match produces two player-events, then compute rolling aggregates shifted so they use only past matches.

In [517]:
events = []
for idx, r in df.iterrows():
    # Winner event
    d = r['tourney_date']
    s = r.get('surface', None)
    events.append({
        'player_name': r['winner_name'],
        'player_id': r['winner_id'],
        'opponent_id': r['loser_id'],
        'date': d,
        'surface': s,
        'is_win': 1,
        'match_id': idx
        # add other stats as needed, e.g. aces, double faults, etc.
    })
    # Loser event
    events.append({
        'player_name': r['loser_name'],
        'player_id': r['loser_id'],
        'opponent_id': r['winner_id'],
        'date': d,
        'surface': s,
        'is_win': 0,
        'match_id': idx
        # add other stats as needed, e.g. aces, double faults, etc.
    })

events_df = pd.DataFrame(events)

events_df = events_df.sort_values('match_id').reset_index(drop=True) 

display(events_df.head())

events_df = events_df.sort_values(['player_id', 'date']).reset_index(drop=True) 

display(events_df.head())


,player_name,player_id,opponent_id,date,surface,is_win,match_id
0,Mikhail Kukushkin,105062,104291,2020-01-06,Hard,1,0
1,Malek Jaziri,104291,105062,2020-01-06,Hard,0,0
2,Pierre Hugues Herbert,105732,106065,2020-01-06,Hard,1,1
3,Marco Cecchinato,106065,105732,2020-01-06,Hard,0,1
4,Laslo Djere,111513,132283,2020-01-06,Hard,1,2


,player_name,player_id,opponent_id,date,surface,is_win,match_id
0,Alexander Zverev,100644,200282,2020-01-06,Hard,0,34
1,Alexander Zverev,100644,133430,2020-01-06,Hard,0,36
2,Alexander Zverev,100644,126774,2020-01-06,Hard,0,38
3,Alexander Zverev,100644,106065,2020-01-20,Hard,1,197
4,Alexander Zverev,100644,106078,2020-01-20,Hard,1,245


Functions to collect data leading up to a match, including:
- Number of wins in previous n matches
- Total matches played
- Total wins
- Total losses

- Total matches on surface
- Total wins on surface
- Total losses on surface

In [518]:
# wins in last n matches before current match
def wins_last_n(events_df, n=5, colname=None):
    if colname is None:
        colname = f'wins_last_{n}'
    # Group by player_id and calculate rolling sum of wins
    events_df[colname] = (events_df.groupby('player_id')['is_win'] # Group by player_id
                            .transform(lambda s: s.rolling(window=n, min_periods=1).sum().shift(1))
                            .fillna(0)
                            .astype(int)
                          )
    return events_df

# total matches played before current match
def total_matches_before(events_df, colname=None):
    if colname is None:
        colname = 'total_matches_before'
    # Group by player_id and calculate rolling count of matches
    events_df[colname] = events_df.groupby('player_id')['match_id'].cumcount()
    return events_df

# total wins before current match
def total_wins_before(events_df, colname=None):
    if colname is None:
        colname = 'total_wins_before'
    # Group by player_id and calculate cumulative sum of wins
    events_df[colname] = (events_df.groupby('player_id')['is_win']
                          .transform(lambda s: s.cumsum().shift(1))
                          .fillna(0)  # Fill NaN with 0 for players with no previous wins
                          .astype(int))  # Convert to integer
    return events_df

# total losses before current match
def total_losses_before(events_df, colname=None):
    if colname is None:
        colname = 'total_losses_before'
    # Group by player_id and calculate cumulative sum of losses
    events_df[colname] = (events_df.groupby('player_id')['is_win']
                          .transform(lambda x: (x == 0).cumsum().shift(1))  # Count losses and shift to use only past matches
                          .fillna(0)  # Fill NaN with 0 for players with no previous losses
                          .astype(int))
    return events_df

# total matches on same surface before current match
def total_matches_on_surface(events_df, colname=None):
    if colname is None:
        colname = 'total_matches_on_surface'
    
    # Create a temporary column to help with grouping
    events_df['player_surface'] = events_df['player_id'].astype(str) + '_' + events_df['surface'].fillna('Unknown')

   
    # Group by player_id and surface, calculate cumulative count of matches
    events_df[colname] = (events_df.groupby('player_surface')['match_id']
                            .cumcount()
                            .fillna(0)  # Fill NaN with 0 for players with no previous matches on that surface
                            .astype(int))
    
    # Drop the temporary column
    events_df.drop('player_surface', axis=1, inplace=True)
    
    return events_df

# wins on same surface before current match
def wins_on_surface(events_df, colname=None):
    if colname is None:
        colname = 'wins_on_surface'
    
    # Create a temporary column to help with grouping
    events_df['player_surface'] = events_df['player_id'].astype(str) + '_' + events_df['surface'].fillna('Unknown')
    
    # Group by player_id and surface, calculate cumulative sum of wins
    events_df[colname] = (events_df.groupby('player_surface')['is_win']
                            .transform(lambda s: s.cumsum().shift(1))
                            .fillna(0)
                            .astype(int))
    
    # Drop the temporary column
    events_df.drop('player_surface', axis=1, inplace=True)
    
    return events_df

# losses on same surface before current match
def losses_on_surface(events_df, colname=None):
    if colname is None:
        colname = 'losses_on_surface'
    
    # Create a temporary column to help with grouping
    events_df['player_surface'] = events_df['player_id'].astype(str) + '_' + events_df['surface'].fillna('Unknown')
    
    # Group by player_id and surface, calculate cumulative sum of losses
    events_df[colname] = (events_df.groupby('player_surface')['is_win']
                            .transform(lambda x: (x == 0).cumsum().shift(1))
                            .fillna(0)
                            .astype(int))
    
    # Drop the temporary column
    events_df.drop('player_surface', axis=1, inplace=True)
    
    return events_df

Run all the functions to update events_df DataFrame

In [519]:
events_df = wins_last_n(events_df, n=5)
events_df = total_matches_before(events_df)
events_df = total_wins_before(events_df)
events_df = total_losses_before(events_df)
events_df = total_matches_on_surface(events_df)
events_df = wins_on_surface(events_df)
events_df = losses_on_surface(events_df)

Merge pre-match rolling columns from events_df back into original df

In [520]:
# Ensure df has a stable match_id that matches what was used when building events (idx in iterrows)
df['match_id'] = df.index
pd.set_option('display.max_columns', None)

# List of feature columns created in events_df (adjust if you add more functions)
feature_cols = [
    'wins_last_5',
    'total_matches_before',
    'total_wins_before',
    'total_losses_before',
    'total_matches_on_surface',
    'wins_on_surface',
    'losses_on_surface'
]

# Subset DF with only what we need
ev = events_df[['match_id', 'player_id', 'is_win'] + feature_cols]

# Split ev into winners and losers (ev contains both winner and loser rows for each match)
w = ev[ev.is_win == 1].copy() # Boolean mask for winners
l = ev[ev.is_win == 0].copy() # Boolean mask for lasers

# Rename feature columns with side prefixes
w.rename(columns={c: f"winner_{c}" for c in feature_cols}, inplace=True)
l.rename(columns={c: f"loser_{c}" for c in feature_cols}, inplace=True)

# Rename player_id columns wiht side prefixes
w.rename(columns={'player_id': 'winner_player_id'}, inplace=True)
l.rename(columns={'player_id': 'loser_player_id'}, inplace=True)

# Drop helper flag
w.drop(columns=['is_win'], inplace=True)
l.drop(columns=['is_win'], inplace=True)

# Merge onto match-level df
df_with_stats = df.merge(w, on='match_id', how='left')
df_with_stats = df_with_stats.merge(l, on='match_id', how='left')

# Sanity check: winner_player_id should equal winner_id etc.
assert (df_with_stats['winner_player_id'] == df['winner_id']).all()
assert (df_with_stats['loser_player_id'] == df['loser_id']).all()


# Show Djokovic only (104925)
pid = 104925
hist = (events_df[events_df['player_id'] == pid]
        .sort_values(by='match_id')
        [['match_id','player_name', 'is_win'] + feature_cols])
#display(hist.head(10))

# Create difference features (winner minus loser) – often useful for modeling
for c in feature_cols:
    df_with_stats[f'diff_{c}'] = df_with_stats[f'winner_{c}'] - df_with_stats[f'loser_{c}']


display(df_with_stats.head())


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,match_id,winner_player_id,winner_wins_last_5,winner_total_matches_before,winner_total_wins_before,winner_total_losses_before,winner_total_matches_on_surface,winner_wins_on_surface,winner_losses_on_surface,loser_player_id,loser_wins_last_5,loser_total_matches_before,loser_total_wins_before,loser_total_losses_before,loser_total_matches_on_surface,loser_wins_on_surface,loser_losses_on_surface,diff_wins_last_5,diff_total_matches_before,diff_total_wins_before,diff_total_losses_before,diff_total_matches_on_surface,diff_wins_on_surface,diff_losses_on_surface
0,2020-0451,Doha,Hard,32,A,2020-01-06,271,105062,NaN,NaN,Mikhail Kukushkin,R,183.0,KAZ,32.0,104291,NaN,WC,Malek Jaziri,R,185.0,TUN,35.9,6-0 6-3,3,R32,68.0,7.0,0.0,51.0,33.0,22.0,12.0,7.0,3.0,3.0,3.0,2.0,44.0,23.0,9.0,10.0,8.0,4.0,9.0,66.0,816.0,229.0,206.0,0,105062,0,0,0,0,0,0,0,104291,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-0451,Doha,Hard,32,A,2020-01-06,272,105732,NaN,NaN,Pierre Hugues Herbert,R,188.0,FRA,28.8,106065,NaN,WC,Marco Cecchinato,R,185.0,ITA,27.2,6-3 6-4,3,R32,71.0,6.0,2.0,51.0,30.0,26.0,12.0,10.0,0.0,1.0,2.0,1.0,59.0,32.0,20.0,13.0,9.0,4.0,7.0,65.0,840.0,75.0,690.0,1,105732,0,0,0,0,0,0,0,106065,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-0451,Doha,Hard,32,A,2020-01-06,273,111513,5.0,NaN,Laslo Djere,R,188.0,SRB,24.5,132283,NaN,NaN,Lorenzo Sonego,R,191.0,ITA,24.6,6-1 3-6 6-2,3,R32,114.0,5.0,2.0,73.0,48.0,33.0,14.0,12.0,4.0,7.0,5.0,3.0,75.0,47.0,28.0,10.0,12.0,4.0,10.0,39.0,1151.0,51.0,990.0,2,111513,0,0,0,0,0,0,0,132283,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-0451,Doha,Hard,32,A,2020-01-06,275,200175,NaN,NaN,Miomir Kecmanovic,R,183.0,SRB,20.3,111442,NaN,NaN,Jordan Thompson,R,183.0,AUS,25.7,6-4 6-2,3,R32,84.0,0.0,1.0,57.0,31.0,23.0,17.0,9.0,0.0,0.0,4.0,1.0,66.0,40.0,26.0,9.0,9.0,7.0,10.0,62.0,881.0,63.0,878.0,3,200175,0,0,0,0,0,0,0,111442,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-0451,Doha,Hard,32,A,2020-01-06,276,117356,NaN,WC,Cem Ilkel,R,185.0,TUR,24.3,105575,NaN,NaN,Ricardas Berankis,R,175.0,LTU,29.5,6-2 4-6 6-2,3,R32,112.0,11.0,1.0,92.0,59.0,42.0,17.0,13.0,4.0,5.0,3.0,3.0,84.0,53.0,35.0,16.0,13.0,5.0,9.0,279.0,144.0,67.0,804.0,4,117356,0,0,0,0,0,0,0,105575,0,0,0,0,0,0,0,0,0,0,0,0,0,0
